In [ ]:
# classify live stream data for a given amount of time
#neg,pos,neu
#store the tweets
my_tweets=list()
result=[0,0,0]

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
pickle_in=open("word_bucket.pickle","rb")
word_bucket=pickle.load(pickle_in)
pickle_in.close()
model_pred=tf.keras.models.load_model("DL_sentimentanalysis_new.h5")
output=["Negative","Positive"]

In [ ]:
def preparing_set(data,vocab):
    dataset=[]
    for desc in data:
        vector=[]
        for d in vocab:
            if d in desc:
                vector.append(1)
            else:
                vector.append(0)
        dataset.append(vector)
    return(dataset)

In [ ]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
def remove_punctuation(string):
    punctuations='''!()[]{};:'"\,<>./?@#$%^&*_~'''
    for chars in string:
        if chars in punctuations:
            string=string.replace(chars,"")
    return(string)

def remove_alpha_numeric(string):
    line=list()
    for words in string.split(" "):
        if words.isalpha() == True and words not in stopwords:
            line.append(words)
    return(line)

def lower_case(string):
    S_list=list()
    for s in string:
        S_list.append(s.lower())     
    return(S_list)

def cleaning_text(data):
    cleaned_data=list()
    for d in data:
        d1=remove_punctuation(d)
        d1=remove_alpha_numeric(d1)
        d1=lower_case(d1)
        cleaned_data.append(d1)
    return cleaned_data

In [ ]:
string=input("Type the hastags whose sentiment want to be analysed")
max_count=int(input("How many english tweets do you wanna consider ?"))
hashtag_list=[string]

In [ ]:
import tweepy
#define a processing class
class Authenticator():
    def authenticate(self,access_info):
        
        #create instance of OAuthHandler class
        auth = tweepy.OAuthHandler(access_info[0],access_info[1])
        
        #use the instancce for setting the access tokens
        auth.set_access_token(access_info[2],access_info[3])
        
        return auth
        

In [ ]:
class Processor():
    def __init__(self):
        #create an object of Authenticator class
        self.twitter_authenticate=Authenticator()
        
    def get_info(self,access_info,hashtag_list):
        
        #authenticate credentials
        auth=self.twitter_authenticate.authenticate(access_info)
        
        #create an instance of API class
        api = tweepy.API(auth)
        
        
        
        #create an instance of MyStreamListener class
        myStreamListener = MyStreamListener()
        
        #create an objects of Stream class, which takes two parameters
        myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
        
        #filter data based on hashtags
        myStream.filter(track=hashtag_list)
        


In [ ]:
import json
#override tweepy.StreamListener class, to handle the tweets.
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self,):
        pass
        
    def on_data(self, tweets):
        tweets_dic = json.loads(tweets)
        #print("NEXT TWEET------>")
        stringg=tweets_dic["text"]
        if len(my_tweets)<max_count:
            string=cleaning_text([stringg])
            test=preparing_set(string,word_bucket)
            test=np.array(test).reshape(-1,len(word_bucket))
            if np.count_nonzero(test)!=0:
                my_tweets.append(stringg)
                prediction=model_pred.predict_proba(test)
                if prediction[0][np.argmax(prediction)]>=0.60:
                    result[np.argmax(prediction)]=result[np.argmax(prediction)]+1
                else:
                    result[2]=result[2]+1
        else:
            return False
        
        return True
        
    def on_error(self, status):
        #for the continuous consumption of data
        if status==420:
            return False
        print(status)

In [ ]:
consumer_key=<paste ur credentials>
consumer_secret=<paste ur credentials>
access_token=<paste ur credentials>
access_token_secret=<paste ur credentials>
access_info=list()
access_info.append(consumer_key)
access_info.append(consumer_secret)
access_info.append(access_token)
access_info.append(access_token_secret)
processor_obj=Processor()
processor_obj.get_info(access_info,hashtag_list)

In [ ]:
#create graph
import matplotlib.pyplot as plt
plt.pie(result,autopct='%1.0f%%',)
plt.legend(["neagtive","positive","neutral"],loc='upper right')
plt.show()

In [ ]:
#if you wanna view the tweets
#for twts in my_tweets:
    #print(twts)